In [ ]:
import array_api_compat.torch as xp
import torch
import parallelproj
from array_api_compat import to_device
import array_api_compat.numpy as np
import matplotlib.pyplot as plt
import time

dict_data = np.load("/home/user/triton/slices_8_lors_40.npy", allow_pickle=True).item()

xstart = xp.asarray(dict_data['xstart']).cuda()
xend = xp.asarray(dict_data['xend']).cuda()
img_origin = xp.asarray(dict_data['img_origin']).cuda()
voxel_size = xp.asarray(dict_data['voxel_size']).cuda()
img_shape = dict_data['img_shape']
x_gt = xp.asarray(dict_data['x']).cuda()
meas = xp.asarray(dict_data['meas']).cuda() + 100

fwd = lambda x: parallelproj.joseph3d_fwd(xstart, xend, x, img_origin, voxel_size)

back = lambda y: parallelproj.joseph3d_back(xstart, xend, img_shape, img_origin, voxel_size, y)


xp.save(fwd(x_gt).cpu(), "parallel_proj_forward.torch")
xp.save(back(xp.asarray(dict_data['meas']).cuda()).cpu(), "parallel_proj_backward.torch")

In [ ]:
tmp_x = xp.ones_like(x_gt)
tmp_y = xp.ones_like(meas)

tmp_1 = (fwd(tmp_x)*tmp_y).sum()
tmp_2 = (back(tmp_y)*tmp_x).sum()
print(tmp_1, tmp_2)